In [7]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, select, TryCast, MetaData
from abc import ABC, abstractmethod
import json
import pyodbc
import pandas as pd
import requests

class Geradora:
    def __init__(self):
        self._data = None
    
    def ReadJson(self):
        with open("./data.json", 'r+') as file:
            self._data = json.load(file)
        return self._data

    def CreateEngine(self):
        data = self.ReadJson()
        conn_url = URL.create(
            "mssql+pyodbc",
            username=data['username'],
            password=data["password"],
            host=data["host"],
            port=1433,
            database=data["database"],
            query={
                "driver": "ODBC Driver 18 for SQL Server",
                "TrustServerCertificate": "yes"
            }
        )
        engine = create_engine(conn_url)
        return engine
    
    def create_query(self, query):
        try:
            dataframe = pd.read_sql_query(query, self.CreateEngine())
            return dataframe
        except:
            print('Erro ao executar a consulta')
        

geradora = Geradora()
dataframe = pd.DataFrame(geradora.create_query("select * from ONE_Bank_Data"))
dataframe

Erro ao executar a consulta


UnboundLocalError: local variable 'dataframe' referenced before assignment

In [ ]:
dataset = dataframe.iloc[:, 3].values

banks = {}
try:
    for i in dataset:
        url = f'https://www.routingnumbers.info/api/name.json?rn={i}'
        response = requests.get(url)
        data = json.loads(response.content)
        if data['message'] == 'OK':
            banks[i] = data['name']
        else:
            pass
except:
    pass

routing = []
bank_name = []
for i in banks:
    routing.append(i)
for i in banks.values():
    bank_name.append(i)
new_dataframe = pd.DataFrame(list(zip(bank_name, routing)), columns=[
                            'Bank', 'Routing'])

df1 = dataframe.merge(new_dataframe, on='Routing')